In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('Advertising.csv')

In [3]:
X = df.drop('sales',axis=1)

In [4]:
y = df['sales']

**Creating multiple features so than we can apply feature scaling on all the features at a time rather than 1st scaling the feature then applying polynomial converter would convert the scaled features**

In [5]:
from sklearn.preprocessing import PolynomialFeatures

In [6]:
polynomial_converter = PolynomialFeatures(degree=3,include_bias=False)

In [7]:
poly_features = polynomial_converter.fit_transform(X)

In [8]:
poly_features.shape

(200, 19)

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
X_train, X_test, y_train, y_test = train_test_split(poly_features, y, test_size=0.3, random_state=101)

In [11]:
X_train.shape

(140, 19)

In [12]:
from sklearn.preprocessing import StandardScaler

In [13]:
scaler = StandardScaler()

In [14]:
scaler.fit(X_train)

StandardScaler()

In [15]:
X_train = scaler.transform(X_train)

In [16]:
X_test = scaler.transform(X_test)

<h1>L2 regression</h1>

In [17]:
from sklearn.linear_model import Ridge

In [18]:
#help(Ridge)

In [19]:
ridge_model = Ridge(alpha = 10)
# Here alpha represents a hyper-parameter lambda

In [20]:
ridge_model.fit(X_train,y_train)

Ridge(alpha=10)

In [21]:
test_predictions = ridge_model.predict(X_test)

In [22]:
from sklearn.metrics import mean_absolute_error,mean_squared_error

In [23]:
MEA = mean_absolute_error(y_test,test_predictions)

In [24]:
MEA

0.5774404204714175

In [25]:
RMSE = np.sqrt(mean_squared_error(y_test,test_predictions))

In [26]:
RMSE

0.8946386461319681

In [27]:
from sklearn.linear_model import RidgeCV

**Ridge Regression with cross-validation, This cross-validation allows to try various alpha values(lambda in formula)**

In [28]:
ridge_cv_model = RidgeCV(alphas=(0.1, 1.0, 10.0),fit_intercept=True,
                         scoring='neg_mean_absolute_error',cv=None,store_cv_values=False)

In [29]:
ridge_cv_model.fit(X_train,y_train)

RidgeCV(scoring='neg_mean_absolute_error')

**As per the concept of the cross validaion some part of the data is the training set and a part of data will be the hold out data set**

In [30]:
ridge_cv_model.alpha_

0.1

In [31]:
from sklearn import metrics

In [32]:
metrics.get_scorer_names()

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'matthews_corrcoef',
 'max_error',
 'mutual_info_score',
 'neg_brier_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_absolute_percentage_error',
 'neg_mean_gamma_deviance',
 'neg_mean_poisson_deviance',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'neg_negative_likelihood_ratio',
 'neg_root_mean_squared_error',
 'normalized_mutual_info_score',
 'positive_likelihood_ratio',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'rand_score',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',

In [33]:
test_predictions = ridge_cv_model.predict(X_test)

In [34]:
MAE = mean_absolute_error(y_test,test_predictions)

In [35]:
MAE

0.42737748843675194

In [36]:
RMSE = np.sqrt(mean_squared_error(y_test,test_predictions))

In [37]:
RMSE

0.6180719926956963

In [38]:
ridge_cv_model.coef_

array([ 5.40769392,  0.5885865 ,  0.40390395, -6.18263924,  4.59607939,
       -1.18789654, -1.15200458,  0.57837796, -0.1261586 ,  2.5569777 ,
       -1.38900471,  0.86059434,  0.72219553, -0.26129256,  0.17870787,
        0.44353612, -0.21362436, -0.04622473, -0.06441449])

In [39]:
ridge_cv_model.best_score_

-0.3749223340292957

<h1>L1 Regression</h1>

In [40]:
from sklearn.linear_model import LassoCV

In [41]:
# lasso_cv_model = LassoCV( eps=0.001,n_alphas=100,alphas=None,
#                     max_iter=1000000,fit_intercept=True,cv=None)

In [42]:
lasso_cv_model = LassoCV(eps=0.1,n_alphas=100,cv=5)

In [43]:
lasso_cv_model.fit(X_train,y_train)

LassoCV(cv=5, eps=0.1)

In [44]:
lasso_cv_model.alpha_

0.4943070909225828

In [45]:
test_predictions = lasso_cv_model.predict(X_test)

In [46]:
MAE = mean_absolute_error(y_test,test_predictions)

In [47]:
RMSE = np.sqrt(mean_squared_error(y_test,test_predictions))

In [48]:
MAE

0.6541723161252856

In [49]:
RMSE

1.1308001022762535

**If we compare these performace matrix values with as that of the values of the Ridge then we will find that Lasso don't perform well**

In [50]:
lasso_cv_model.coef_

array([1.002651  , 0.        , 0.        , 0.        , 3.79745279,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        ])

<h1>Elastic Net</h1>

In [51]:
from sklearn.linear_model import ElasticNetCV

In [52]:
elastic_model = ElasticNetCV(l1_ratio=[.1, .5, .7, .9, .95, .99, 1],eps=0.001,
n_alphas=100,max_iter=1000000,alphas=None,fit_intercept=True)

In [53]:
elastic_model.fit(X_train,y_train)

ElasticNetCV(l1_ratio=[0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1], max_iter=1000000)

In [54]:
elastic_model.l1_ratio_

1.0

In [55]:
elastic_model.alpha_

0.004943070909225827

In [56]:
test_predictions = elastic_model.predict(X_test)

In [57]:
MAE = mean_absolute_error(y_test,test_predictions)

In [58]:
MAE

0.4335034618590069